In [9]:
import requests
from bs4 import BeautifulSoup

In [10]:
link_get_data_nike = 'https://www.nike.com/w/mens-shoes-nik1zy7ok'
response = requests.get(link_get_data_nike)

soup = BeautifulSoup(response.text, 'html.parser')

# Get all products
products = soup.find_all('div', class_='product-card__body')


In [20]:
for product in products[0]:
    name_product = product.find('div', class_='product-card__title').text
    price = product.find('div', class_='product-price').text
    img_product = product.find('img', class_='product-card__hero-image css-1fxh5tw')
    product_detail_link = product.find('a', class_='product-card__link-overlay').get('href')
    response_product_detail = requests.get(product_detail_link)
    soup_product_detail = BeautifulSoup(response_product_detail.text, 'html.parser')
    test = soup_product_detail.find_all('label')
    print(test)
    product_detail = soup_product_detail.find('div', class_='description-preview').text
    colors_product_detail = soup_product_detail.find('li', class_='description-preview__color-description ncss-li')
    sizes_product_detail = soup_product_detail.find_all('label', class_='css-xf3ahq')

[<label class="ncss-label pl7-sm u-bold" data-sq-exempt="true" data-var="functionalLabel" for="PreCookie-functional" id="PreCookie-functional-label"><span data-sq-exempt="true" id="hf_cookie_label_functional">Functional</span></label>, <label class="ncss-label pl7-sm u-bold" data-sq-exempt="true" data-var="allowPerformanceLabel" for="PreCookie-allowPerformance" id="PreCookie-allowPerformance-label"><span data-sq-exempt="true" id="hf_cookie_label_performance">Performance</span></label>, <label class="ncss-label pl7-sm u-bold" data-sq-exempt="true" data-var="allowMarketingLabel" for="PreCookie-allowMarketing" id="PreCookie-allowMarketing-label"><span data-sq-exempt="true" id="hf_cookie_label_socialMedia">Social Media and Advertising</span></label>, <label class="css-15juft3" for="More Reviews">More Reviews</label>]


####
Crawl data
###

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options as ChromeOptions
import time

# Configure Chrome options
chrome_options = ChromeOptions()
# Run Chrome in headless mode (no GUI)
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Set up the Chrome web driver
driver = webdriver.Chrome(service=ChromeService(
    "./chromedriver.exe"), options=chrome_options)

# URL of the web page you want to scrape
url = 'https://www.nike.com/vn/w/mens-shoes-nik1zy7ok'

# Open the web page
driver.get(url)

# Scroll down to load more content until no more items are loaded
scroll_delay = 2  # Adjust this value based on your page's loading speed
scroll_count = 0
max_scroll_attempts = 10  # Maximum number of times to scroll
loaded_items = set()

# while scroll_count < max_scroll_attempts:
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(scroll_delay)
#     new_items = driver.find_elements(By.TAG_NAME, 'img')
#     print(len(new_items))

#     # Check if new items have been loaded, and if not, break the loop
#     new_items_count = len(loaded_items)
#     loaded_items.update(new_items)
#     if len(loaded_items) == new_items_count:
#         break

#     scroll_count += 1

# Scroll down to load more images (you may need to customize this based on the website)
for _ in range(10):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)  # Wait for the page to load

last_height = driver.execute_script("return document.body.scrollHeight")

# while scroll_count < max_scroll_attempts:
#     # Record the current number of product elements

#     # Scroll down
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     # Wait to load page
#     time.sleep(10)

#     # Calculate new scroll height and compare with last scroll height
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     print("new_height", new_height)
#     print("last_height", last_height)
#     if new_height == last_height:
#         break
#     last_height = new_height


#     # Find the updated list of product elements
#     product_elements = driver.find_elements(By.CLASS_NAME, 'product-card__body')

#     # If the number of product elements has not increased, break the loop
#     print("len(product_elements)", len(product_elements))
#     # if len(product_elements) == current_product_count:
#     #     break

#     scroll_count += 1

# Find all the image elements on the page
product_elements = driver.find_elements(By.CLASS_NAME, 'product-card__body')

print("Number of products:", len(product_elements))
output = []
for product_element in product_elements[:2]:
    product_img = product_element.find_element(
        By.TAG_NAME, 'img').get_attribute('src')
    print("product_img", product_img)
    product_name = product_element.find_element(
        By.CLASS_NAME, 'product-card__title').text
    print("product_name", product_name)
    product_price = product_element.find_element(
        By.CLASS_NAME, 'product-price').text
    print("product_price", product_price)
    product_link_detail = product_element.find_element(
        By.CLASS_NAME, 'product-card__link-overlay').get_attribute('href')
    print("product_link_detail", product_link_detail)
    driver_detail = webdriver.Chrome(service=ChromeService(
        "./chromedriver.exe"), options=chrome_options)
    driver_detail.get(product_link_detail)
    time.sleep(5)
    product_detail = driver_detail.find_element(
        By.CLASS_NAME, 'description-preview').text
    product_color = driver_detail.find_element(
        By.CLASS_NAME, 'description-preview__color-description').text
    product_size_elements = driver_detail.find_elements(
        By.CLASS_NAME, 'css-xf3ahq')
    product_sizes = [test.text for test in product_size_elements]
    imgs_elements = driver_detail.find_elements(By.TAG_NAME, 'img')
    imgs_related = []
    for img_element in imgs_elements:
        src_img_element = img_element.get_attribute('src')
        string_loading_img = 'LOADING'
        string_source_from_nike = 'https://static.nike.com'
        if string_source_from_nike not in src_img_element:
            continue
        if string_loading_img not in src_img_element:
            print("src_img_element", src_img_element)
            imgs_related.append(src_img_element)

    print("product_detail", product_detail)
    print("product_color", product_color)
    print("product_sizes", product_sizes)
    each_product_data = {
        'product_img': product_img,
        'product_name': product_name,
        'product_price': product_price,
        'product_link_detail': product_link_detail,
        'product_detail': product_detail,
        'product_color': product_color,
        'product_sizes': product_sizes,
        'imgs_related': imgs_related
    }
    output.append(each_product_data)


    driver_detail.quit()

import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=4)

# Close the web driver
driver.quit()


Number of products: 48
product_img https://static.nike.com/a/images/c_limit,w_592,f_auto/t_product_v1/e6da41fa-1be4-4ce5-b89c-22be4f1f02d4/air-force-1-07-shoes-WrLlWX.png
product_name Nike Air Force 1 '07
product_price 2,929,000₫
product_link_detail https://www.nike.com/vn/t/air-force-1-07-shoes-WrLlWX/CW2288-111
src_img_element https://static.nike.com/a/images/t_PDP_1280_v1/f_auto,q_auto:eco/b7d9211c-26e7-431a-ac24-b0540fb3c00f/air-force-1-07-shoes-WrLlWX.png
src_img_element https://static.nike.com/a/images/t_PDP_1280_v1/f_auto,q_auto:eco/00375837-849f-4f17-ba24-d201d27be49b/air-force-1-07-shoes-WrLlWX.png
src_img_element https://static.nike.com/a/images/t_PDP_1280_v1/f_auto,q_auto:eco/3cc96f43-47b6-43cb-951d-d8f73bb2f912/air-force-1-07-shoes-WrLlWX.png
src_img_element https://static.nike.com/a/images/t_PDP_1280_v1/f_auto,q_auto:eco/33533fe2-1157-4001-896e-1803b30659c8/air-force-1-07-shoes-WrLlWX.png
src_img_element https://static.nike.com/a/images/t_PDP_1280_v1/f_auto,q_auto:eco/a0a3

KeyboardInterrupt: 